#Carga de datos
Partimos de la exisitencia de un archivo .csv extraido de esta [pagina](https://www.kaggle.com/amritvirsinghx/web-series-ultimate-edition).

Cargamos el csv en una variable pandas


In [34]:
import pandas
data = pandas.read_csv('All_Streaming_Shows.csv')

Como vamos a utilizar la descripción para aplicar el algoritmo de recomendación, las series sin descripción no nos son útiles, por lo que las borramos, y reseteamos el indice para poder manipular las filas más adelante.

In [35]:
data = data[data['Description'] != '-1']
data = data.reset_index()
data

,index,Series Title,Year Released,Content Rating,IMDB Rating,R Rating,Genre,Description,No of Seasons,Streaming Platform
0,0,Breaking Bad,2008,18+,9.5,100,"Crime,Drama","When Walter White, a New Mexico chemistry teac...",5Seasons,Netflix
1,1,Game of Thrones,2011,18+,9.3,99,"Action & Adventure,Drama",Seven noble families fight for control of the ...,8Seasons,"HBO MAX,HBO"
2,2,Rick and Morty,2013,18+,9.2,97,"Animation,Comedy",Rick is a mentally-unbalanced but scientifical...,4Seasons,"Free Services,HBO MAX,Hulu"
3,3,Stranger Things,2016,16+,8.8,96,"Drama,Fantasy","When a young boy vanishes, a small town uncove...",3Seasons,Netflix
4,4,The Boys,2019,18+,8.7,95,"Action & Adventure,Comedy",A group of vigilantes known informally as “The...,2Seasons,Prime Video
...,...,...,...,...,...,...,...,...,...,...
11869,12344,"Stop, Breathe & Think Kids: Mindful Games",2017,NaN,NaN,-1,"2017,Hulu",Mindfulness made easy and fun for kids. Discov...,1Season,Hulu
11870,12348,A Fishing Story with Ronnie Green,2017,NaN,NaN,-1,"2017,Prime Video",A Fishing Story with Ronnie Green has one or m...,2Seasons,"Prime Video,fuboTV"
11871,12350,NHL Road to the Outdoor Classics,2016,NaN,NaN,-1,"2016,Prime Video",Road to the NHL Outdoor Classics takes us deep...,1Season,"Prime Video,Epix"
11872,12351,Addy Media,2018,NaN,NaN,-1,"2018,Prime Video",Addy Media has one or more episodes streaming ...,1Season,Prime Video


Vemos que para el tipo de dato 'Content Rating' existen varios posibles valores, y el valor "nan", como esto es un dato nulo, vamos a asumir que todas las series con valor "nan" son para todos los publicos, y vamos a subsituirlo por el valor "all"

In [36]:
data['Content Rating'].unique()

array(['18+', '16+', '7+', 'all', nan, '13+'], dtype=object)

In [37]:
data['Content Rating'] = data['Content Rating'].fillna('all')
data

,index,Series Title,Year Released,Content Rating,IMDB Rating,R Rating,Genre,Description,No of Seasons,Streaming Platform
0,0,Breaking Bad,2008,18+,9.5,100,"Crime,Drama","When Walter White, a New Mexico chemistry teac...",5Seasons,Netflix
1,1,Game of Thrones,2011,18+,9.3,99,"Action & Adventure,Drama",Seven noble families fight for control of the ...,8Seasons,"HBO MAX,HBO"
2,2,Rick and Morty,2013,18+,9.2,97,"Animation,Comedy",Rick is a mentally-unbalanced but scientifical...,4Seasons,"Free Services,HBO MAX,Hulu"
3,3,Stranger Things,2016,16+,8.8,96,"Drama,Fantasy","When a young boy vanishes, a small town uncove...",3Seasons,Netflix
4,4,The Boys,2019,18+,8.7,95,"Action & Adventure,Comedy",A group of vigilantes known informally as “The...,2Seasons,Prime Video
...,...,...,...,...,...,...,...,...,...,...
11869,12344,"Stop, Breathe & Think Kids: Mindful Games",2017,all,NaN,-1,"2017,Hulu",Mindfulness made easy and fun for kids. Discov...,1Season,Hulu
11870,12348,A Fishing Story with Ronnie Green,2017,all,NaN,-1,"2017,Prime Video",A Fishing Story with Ronnie Green has one or m...,2Seasons,"Prime Video,fuboTV"
11871,12350,NHL Road to the Outdoor Classics,2016,all,NaN,-1,"2016,Prime Video",Road to the NHL Outdoor Classics takes us deep...,1Season,"Prime Video,Epix"
11872,12351,Addy Media,2018,all,NaN,-1,"2018,Prime Video",Addy Media has one or more episodes streaming ...,1Season,Prime Video


El valor de no of seasons es una cadena de texto, nos interesa más convertirla en un valor entero, por lo que iteramos por todo el dataset, convirtiendo cada valor en un entero en una nueva columna. (Para esta operacion necesitamos que los índices estén consecutivos).

In [38]:
import re
p = re.compile('-?[0-9]+')
var = 0
data['Number of Seasons'] = 0
for row in data.itertuples():
  data['Number of Seasons'][var] = int(p.match(row[9]).group())
  var +=1
print(var)
data


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


11874


,index,Series Title,Year Released,Content Rating,IMDB Rating,R Rating,Genre,Description,No of Seasons,Streaming Platform,Number of Seasons
0,0,Breaking Bad,2008,18+,9.5,100,"Crime,Drama","When Walter White, a New Mexico chemistry teac...",5Seasons,Netflix,5
1,1,Game of Thrones,2011,18+,9.3,99,"Action & Adventure,Drama",Seven noble families fight for control of the ...,8Seasons,"HBO MAX,HBO",8
2,2,Rick and Morty,2013,18+,9.2,97,"Animation,Comedy",Rick is a mentally-unbalanced but scientifical...,4Seasons,"Free Services,HBO MAX,Hulu",4
3,3,Stranger Things,2016,16+,8.8,96,"Drama,Fantasy","When a young boy vanishes, a small town uncove...",3Seasons,Netflix,3
4,4,The Boys,2019,18+,8.7,95,"Action & Adventure,Comedy",A group of vigilantes known informally as “The...,2Seasons,Prime Video,2
...,...,...,...,...,...,...,...,...,...,...,...
11869,12344,"Stop, Breathe & Think Kids: Mindful Games",2017,all,NaN,-1,"2017,Hulu",Mindfulness made easy and fun for kids. Discov...,1Season,Hulu,1
11870,12348,A Fishing Story with Ronnie Green,2017,all,NaN,-1,"2017,Prime Video",A Fishing Story with Ronnie Green has one or m...,2Seasons,"Prime Video,fuboTV",2
11871,12350,NHL Road to the Outdoor Classics,2016,all,NaN,-1,"2016,Prime Video",Road to the NHL Outdoor Classics takes us deep...,1Season,"Prime Video,Epix",1
11872,12351,Addy Media,2018,all,NaN,-1,"2018,Prime Video",Addy Media has one or more episodes streaming ...,1Season,Prime Video,1


Los datos de puntuación de imdb y R vamos a realizar una media con ellos. Como existen datos NaN para el IMDB tenemos que normalizarlos, en este caso lo ponemos a -1 ya que es el valor que tiene R rating para sus vacios. Como las columnas de imdb y r rating tienen distinta escala, tenemos que poner ambos en la misma escala, en este caso sobre 100 puntos. Una vez tenemos las columnas en el mismo formato, podemos generar una nueva columna con la media de ambas.

In [39]:
data['IMDB Rating'] = data['IMDB Rating'] * 10
data['IMDB Rating'] = data['IMDB Rating'].fillna(-1)
data['Mean Rating'] = data[['IMDB Rating', 'R Rating']].mean(axis=1)
data

,index,Series Title,Year Released,Content Rating,IMDB Rating,R Rating,Genre,Description,No of Seasons,Streaming Platform,Number of Seasons,Mean Rating
0,0,Breaking Bad,2008,18+,95.0,100,"Crime,Drama","When Walter White, a New Mexico chemistry teac...",5Seasons,Netflix,5,97.5
1,1,Game of Thrones,2011,18+,93.0,99,"Action & Adventure,Drama",Seven noble families fight for control of the ...,8Seasons,"HBO MAX,HBO",8,96.0
2,2,Rick and Morty,2013,18+,92.0,97,"Animation,Comedy",Rick is a mentally-unbalanced but scientifical...,4Seasons,"Free Services,HBO MAX,Hulu",4,94.5
3,3,Stranger Things,2016,16+,88.0,96,"Drama,Fantasy","When a young boy vanishes, a small town uncove...",3Seasons,Netflix,3,92.0
4,4,The Boys,2019,18+,87.0,95,"Action & Adventure,Comedy",A group of vigilantes known informally as “The...,2Seasons,Prime Video,2,91.0
...,...,...,...,...,...,...,...,...,...,...,...,...
11869,12344,"Stop, Breathe & Think Kids: Mindful Games",2017,all,-1.0,-1,"2017,Hulu",Mindfulness made easy and fun for kids. Discov...,1Season,Hulu,1,-1.0
11870,12348,A Fishing Story with Ronnie Green,2017,all,-1.0,-1,"2017,Prime Video",A Fishing Story with Ronnie Green has one or m...,2Seasons,"Prime Video,fuboTV",2,-1.0
11871,12350,NHL Road to the Outdoor Classics,2016,all,-1.0,-1,"2016,Prime Video",Road to the NHL Outdoor Classics takes us deep...,1Season,"Prime Video,Epix",1,-1.0
11872,12351,Addy Media,2018,all,-1.0,-1,"2018,Prime Video",Addy Media has one or more episodes streaming ...,1Season,Prime Video,1,-1.0


## El formato de la columna Streaming Platform está muy especificado por eso simplemente vamos a dejar como plataformas posibles:
    -Amazon Prime Video
    -Netflix
    -HBO
    -y Others, donde se ubicarán el resto


Primero creamos las columnas correspondientes a estos atributos dummies:




In [40]:
data.insert(9,'Others',0)
data.insert(9,'Prime Video',0)
data.insert(9,'HBO',0)
data.insert(9,'Netflix',0)

data

,index,Series Title,Year Released,Content Rating,IMDB Rating,R Rating,Genre,Description,No of Seasons,Netflix,HBO,Prime Video,Others,Streaming Platform,Number of Seasons,Mean Rating
0,0,Breaking Bad,2008,18+,95.0,100,"Crime,Drama","When Walter White, a New Mexico chemistry teac...",5Seasons,0,0,0,0,Netflix,5,97.5
1,1,Game of Thrones,2011,18+,93.0,99,"Action & Adventure,Drama",Seven noble families fight for control of the ...,8Seasons,0,0,0,0,"HBO MAX,HBO",8,96.0
2,2,Rick and Morty,2013,18+,92.0,97,"Animation,Comedy",Rick is a mentally-unbalanced but scientifical...,4Seasons,0,0,0,0,"Free Services,HBO MAX,Hulu",4,94.5
3,3,Stranger Things,2016,16+,88.0,96,"Drama,Fantasy","When a young boy vanishes, a small town uncove...",3Seasons,0,0,0,0,Netflix,3,92.0
4,4,The Boys,2019,18+,87.0,95,"Action & Adventure,Comedy",A group of vigilantes known informally as “The...,2Seasons,0,0,0,0,Prime Video,2,91.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11869,12344,"Stop, Breathe & Think Kids: Mindful Games",2017,all,-1.0,-1,"2017,Hulu",Mindfulness made easy and fun for kids. Discov...,1Season,0,0,0,0,Hulu,1,-1.0
11870,12348,A Fishing Story with Ronnie Green,2017,all,-1.0,-1,"2017,Prime Video",A Fishing Story with Ronnie Green has one or m...,2Seasons,0,0,0,0,"Prime Video,fuboTV",2,-1.0
11871,12350,NHL Road to the Outdoor Classics,2016,all,-1.0,-1,"2016,Prime Video",Road to the NHL Outdoor Classics takes us deep...,1Season,0,0,0,0,"Prime Video,Epix",1,-1.0
11872,12351,Addy Media,2018,all,-1.0,-1,"2018,Prime Video",Addy Media has one or more episodes streaming ...,1Season,0,0,0,0,Prime Video,1,-1.0


## Transformamos los "-1" que encontremos en "Not Found" para que podamos trabajar con Strings solamente.

In [41]:
data['Streaming Platform'] = data['Streaming Platform'].fillna("Not Found")
data1 = data[data["Streaming Platform"] == "-1"]
data1["Streaming Platform"] = "Not Found" 
data[data["Streaming Platform"] == "-1"] = data1
data

,index,Series Title,Year Released,Content Rating,IMDB Rating,R Rating,Genre,Description,No of Seasons,Netflix,HBO,Prime Video,Others,Streaming Platform,Number of Seasons,Mean Rating
0,0,Breaking Bad,2008,18+,95.0,100,"Crime,Drama","When Walter White, a New Mexico chemistry teac...",5Seasons,0,0,0,0,Netflix,5,97.5
1,1,Game of Thrones,2011,18+,93.0,99,"Action & Adventure,Drama",Seven noble families fight for control of the ...,8Seasons,0,0,0,0,"HBO MAX,HBO",8,96.0
2,2,Rick and Morty,2013,18+,92.0,97,"Animation,Comedy",Rick is a mentally-unbalanced but scientifical...,4Seasons,0,0,0,0,"Free Services,HBO MAX,Hulu",4,94.5
3,3,Stranger Things,2016,16+,88.0,96,"Drama,Fantasy","When a young boy vanishes, a small town uncove...",3Seasons,0,0,0,0,Netflix,3,92.0
4,4,The Boys,2019,18+,87.0,95,"Action & Adventure,Comedy",A group of vigilantes known informally as “The...,2Seasons,0,0,0,0,Prime Video,2,91.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11869,12344,"Stop, Breathe & Think Kids: Mindful Games",2017,all,-1.0,-1,"2017,Hulu",Mindfulness made easy and fun for kids. Discov...,1Season,0,0,0,0,Hulu,1,-1.0
11870,12348,A Fishing Story with Ronnie Green,2017,all,-1.0,-1,"2017,Prime Video",A Fishing Story with Ronnie Green has one or m...,2Seasons,0,0,0,0,"Prime Video,fuboTV",2,-1.0
11871,12350,NHL Road to the Outdoor Classics,2016,all,-1.0,-1,"2016,Prime Video",Road to the NHL Outdoor Classics takes us deep...,1Season,0,0,0,0,"Prime Video,Epix",1,-1.0
11872,12351,Addy Media,2018,all,-1.0,-1,"2018,Prime Video",Addy Media has one or more episodes streaming ...,1Season,0,0,0,0,Prime Video,1,-1.0


## Por último recoreremos la columna que contiene la información de las plataformas que ofrecen cada serie y si encaja con alguna de nuestras plataformas elegidas le pondremos un 1 en su variable dummie correspondiente.

In [47]:
aux = data["Streaming Platform"]
for i in range(aux.size):
      if 'Netflix' in aux[i]:
          data['Netflix'][i] = 1
      if 'Prime Video' in aux[i]:
          data['Prime Video'][i] = 1
      if 'HBO' in aux[i]:
          data['HBO'][i] = 1
      if(not('Netflix' in aux[i])
         and not('Prime Video' in aux[i])
         and not('HBO' in aux[i])):
          data['Others'][i] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: Se

##Y ya tenemos los datos listos para alimentar el algoritmo

In [48]:
data

,index,Series Title,Year Released,Content Rating,IMDB Rating,R Rating,Genre,Description,No of Seasons,Netflix,HBO,Prime Video,Others,Streaming Platform,Number of Seasons,Mean Rating
0,0,Breaking Bad,2008,18+,95.0,100,"Crime,Drama","When Walter White, a New Mexico chemistry teac...",5Seasons,1,0,0,0,Netflix,5,97.5
1,1,Game of Thrones,2011,18+,93.0,99,"Action & Adventure,Drama",Seven noble families fight for control of the ...,8Seasons,0,1,0,0,"HBO MAX,HBO",8,96.0
2,2,Rick and Morty,2013,18+,92.0,97,"Animation,Comedy",Rick is a mentally-unbalanced but scientifical...,4Seasons,0,1,0,0,"Free Services,HBO MAX,Hulu",4,94.5
3,3,Stranger Things,2016,16+,88.0,96,"Drama,Fantasy","When a young boy vanishes, a small town uncove...",3Seasons,1,0,0,0,Netflix,3,92.0
4,4,The Boys,2019,18+,87.0,95,"Action & Adventure,Comedy",A group of vigilantes known informally as “The...,2Seasons,0,0,1,0,Prime Video,2,91.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11869,12344,"Stop, Breathe & Think Kids: Mindful Games",2017,all,-1.0,-1,"2017,Hulu",Mindfulness made easy and fun for kids. Discov...,1Season,0,0,0,1,Hulu,1,-1.0
11870,12348,A Fishing Story with Ronnie Green,2017,all,-1.0,-1,"2017,Prime Video",A Fishing Story with Ronnie Green has one or m...,2Seasons,0,0,1,0,"Prime Video,fuboTV",2,-1.0
11871,12350,NHL Road to the Outdoor Classics,2016,all,-1.0,-1,"2016,Prime Video",Road to the NHL Outdoor Classics takes us deep...,1Season,0,0,1,0,"Prime Video,Epix",1,-1.0
11872,12351,Addy Media,2018,all,-1.0,-1,"2018,Prime Video",Addy Media has one or more episodes streaming ...,1Season,0,0,1,0,Prime Video,1,-1.0
